# **Task 3 - Modeling**

## **Section 1 - Setup**

In [20]:
!pip install optuna pyspark -qqq
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import optuna
from pyspark.sql import SparkSession

## **Section 2 - Data loading**

In [21]:
spark = SparkSession.builder.appName('temp').getOrCreate()

# read datasets
sales_df = pd.read_csv(f"sales.csv")
sales_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
stock_df = pd.read_csv(f"sensor_stock_levels.csv")
stock_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')
temp_df = pd.read_csv(f"sensor_storage_temperature.csv")
temp_df.drop(columns=["Unnamed: 0"], inplace=True, errors='ignore')

def convert_to_datetime(data: pd.DataFrame = None, column: str = None):
  dummy = data.copy()
  dummy[column] = pd.to_datetime(dummy[column], format='%Y-%m-%d %H:%M:%S')
  return dummy

# convert str to datetime
sales_df = convert_to_datetime(sales_df, 'timestamp')
stock_df = convert_to_datetime(stock_df, 'timestamp')
temp_df = convert_to_datetime(temp_df, 'timestamp')

from datetime import datetime

# helper function to convert datetime to desired format
def convert_timestamp_to_hourly(data: pd.DataFrame = None, column: str = None):
  dummy = data.copy()
  new_ts = dummy[column].tolist()
  new_ts = [i.strftime('%Y-%m-%d %H:00:00') for i in new_ts]
  new_ts = [datetime.strptime(i, '%Y-%m-%d %H:00:00') for i in new_ts]
  dummy[column] = new_ts
  return dummy

# convert datetime to hour approximation
sales_df = convert_timestamp_to_hourly(sales_df, 'timestamp')
stock_df = convert_timestamp_to_hourly(stock_df, 'timestamp')
temp_df = convert_timestamp_to_hourly(temp_df, 'timestamp')

print(sales_df.head())

# aggregate data based on time & product ID
# total sales & mean estimated stock percentage
# add temperature aggregations for timestamp
sales_agg = sales_df.groupby(['timestamp', 'product_id']).agg({'quantity': 'sum'}).reset_index()
stock_agg = stock_df.groupby(['timestamp', 'product_id']).agg({'estimated_stock_pct': 'mean'}).reset_index()
temp_agg = temp_df.groupby(['timestamp']).agg({'temperature': 'mean'}).reset_index()

merged_df = stock_agg.merge(sales_agg, on=['timestamp', 'product_id'], how='left')
merged_df = merged_df.merge(temp_agg, on='timestamp', how='left')
print(merged_df.info())


merged_df['quantity'] = merged_df['quantity'].fillna(0)
merged_df = merged_df.dropna()

# add features to aggregated dataframe
product_categories = sales_df[['product_id', 'category']]
product_categories = product_categories.drop_duplicates()
product_price = sales_df[['product_id', 'unit_price']]
product_price = product_price.drop_duplicates()
merged_df = merged_df.merge(product_categories, on="product_id", how="left")
merged_df = merged_df.merge(product_price, on="product_id", how="left")

# add time based features
merged_df['day'] = merged_df['timestamp'].dt.day
merged_df['dow'] = merged_df['timestamp'].dt.dayofweek
merged_df['hour'] = merged_df['timestamp'].dt.hour
merged_df.drop(columns=['timestamp'], inplace=True)

                         transaction_id           timestamp  \
0  a1c82654-c52c-45b3-8ce8-4c2a1efe63ed 2022-03-02 09:00:00   
1  931ad550-09e8-4da6-beaa-8c9d17be9c60 2022-03-06 10:00:00   
2  ae133534-6f61-4cd6-b6b8-d1c1d8d90aea 2022-03-04 17:00:00   
3  157cebd9-aaf0-475d-8a11-7c8e0f5b76e4 2022-03-02 17:00:00   
4  a81a6cd3-5e0c-44a2-826c-aea43e46c514 2022-03-05 14:00:00   

                             product_id category customer_type  unit_price  \
0  3bc6c1ea-0198-46de-9ffd-514ae3338713    fruit          gold        3.99   
1  ad81b46c-bf38-41cf-9b54-5fe7f5eba93e    fruit      standard        3.99   
2  7c55cbd4-f306-4c04-a030-628cbe7867c1    fruit       premium        0.19   
3  80da8348-1707-403f-8be7-9e6deeccc883    fruit          gold        0.19   
4  7f5e86e6-f06f-45f6-bf44-27b095c9ad1d    fruit         basic        4.49   

   quantity  total payment_type  
0         2   7.98     e-wallet  
1         1   3.99     e-wallet  
2         2   0.38     e-wallet  
3         4   0.

In [22]:
import plotly.express as px

# numerical columns
c = merged_df.select_dtypes(include=['int','float']).columns
numerical = list(c)
numerical.remove('estimated_stock_pct')

px.box(merged_df[c],y=merged_df[c].columns,template='plotly_white',
       width=600,height=400,title='univariate feature distribution')

In [23]:
from scipy.stats import skew,kurtosis

skew_data = merged_df[c].apply(lambda x: skew(x),axis=0)
skew_data

estimated_stock_pct    0.015307
quantity               2.417739
temperature            0.137318
unit_price             0.495049
day                    0.001889
dow                    0.000616
hour                   0.001924
dtype: float64

In [24]:
from sklearn.preprocessing import PowerTransformer,QuantileTransformer
from sklearn.preprocessing import MaxAbsScaler,Normalizer
import plotly.express as px

# column transformations
def log_column(df,columns):
    df[columns] = df[columns].apply(lambda x: np.log(x + 1))
    return df

merged_df_tr = log_column(merged_df,'quantity')
print(merged_df_tr.shape)

px.box(merged_df_tr[c],
       y=merged_df[c].columns,
       template='plotly_white',
       width=600,height=400,
       title='univariate feature distribution')

(10845, 9)


In [25]:
skew_data = merged_df[c].apply(lambda x: skew(x),axis=0)
skew_data

estimated_stock_pct    0.015307
quantity               1.374435
temperature            0.137318
unit_price             0.495049
day                    0.001889
dow                    0.000616
hour                   0.001924
dtype: float64

In [26]:
from sklearn.preprocessing import MaxAbsScaler,Normalizer

# column normalisation
def normalise_columns(df,columns,norm_id):
    normaliser = Normalizer(norm=norm_id)
    df[columns] = normaliser.fit_transform(df[columns])
    return df

# transformation & minmax
merged_df_tr_minmax = normalise_columns(merged_df_tr,numerical,'max')

In [27]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import seaborn as sns

# encoding transformations
le=LabelEncoder()
merged_df_tr['category']=le.fit_transform(merged_df_tr['category'])
merged_df_tr_minmax['category']=le.fit_transform(merged_df_tr_minmax['category'])

merged_df_tr_ohe = pd.get_dummies(merged_df_tr, columns=['category'])

In [28]:
from sklearn.model_selection import train_test_split as tts

def split_data(df):

    # remove product_id, category
    df = df[df.columns.difference(['product_id','category'])]
    y=df.estimated_stock_pct
    x=df.drop(columns='estimated_stock_pct')

    xtrain , xtest, ytrain, ytest = tts(x,y,shuffle=True,train_size=.75)
    print(f"xtrain: {xtrain.shape} and xtest: {xtest.shape}")
    print(f"ytrain: {ytrain.shape} and ytest: {ytest.shape}")
    return xtrain,xtest,ytrain,ytest

In [49]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,median_absolute_error
from sklearn.model_selection import KFold,cross_val_score,RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.linear_model import ARDRegression
from sklearn.ensemble import AdaBoostRegressor,HistGradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
import pickle as pk
import warnings
warnings.filterwarnings('ignore')

# randomised gridsearch cross validation
# 7 kfolds

def p(g,model,name):
    pt=RandomizedSearchCV(estimator=model,cv=7,
                          param_distributions=g,
                          n_jobs=-1,
                          random_state=344)

    pt.fit(xtrain,ytrain)
    best = pt.best_estimator_
    best.fit(xtrain,ytrain)
    ypred = best.predict(xtest)

    pk.dump(best, open(f'"{name}.pkl"', 'wb'))
    return name,mean_absolute_error(ytest,ypred)

def pt(model,name):
    model.fit(xtrain,ytrain)
    ypred = model.predict(xtest)
    pk.dump(model, open(f'"{name}.pkl"', 'wb'))
    return name,mean_absolute_error(ytest,ypred),model

In [30]:
def check_models():

    results = {'name':[],'mae':[]}

    # histogram gradient boosting
    h=HistGradientBoostingRegressor(random_state=34563,
                                    max_bins=244,
                                    max_depth=30)

    g={'learning_rate':[0.1,0.01],
    'max_iter':[100,200,500,600,800,900],
    'max_leaf_nodes':[20,30],
    'l2_regularization':[1,0.01],
    'tol':[1e-7,1e-8]}

    name,mae = p(g,h,'histgrdbstreg')
    results['name'].append(name); results['mae'].append(mae)

    # adaboost
    ada=AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=16))

    grid={'n_estimators':[7,8,10],
        'learning_rate':[1.2,1.6,2],
        'loss':['linear', 'square', 'exponential']}

    name,mae = p(grid,ada,'adabstreg')
    results['name'].append(name); results['mae'].append(mae)

    # bagging regressor
    bag=BaggingRegressor(estimator=DecisionTreeRegressor(max_depth=17),
                        oob_score=False,
                        n_jobs=-1)

    bgrid={'n_estimators':[10,13,16]}
    name, mae = p(bgrid,bag,'bagreg')
    results['name'].append(name); results['mae'].append(mae)

    # random forest
    r=RandomForestRegressor(n_jobs=-1,oob_score=True)

    rgrid={'max_depth':[170,190,200,210],
        'max_features':['sqrt', 'log2'],
        'max_samples':[30,100,150,200],
        'max_leaf_nodes':[20,40,60,100]}

    name,mae = p(rgrid,r,'randfrstreg')
    results['name'].append(name); results['mae'].append(mae)

    r=RandomForestRegressor(n_jobs=-1,oob_score=True)

    # Stochastic Gradient Regressor
    sgd=SGDRegressor()
    sgdg={'penalty':['l2', 'l1', 'elasticnet', None],
        'max_iter':[100,400,800],
        'tol':[1e-3,1e-5,1e-8],
        'alpha':[0.1,.001,0.0001,1],
        'learning_rate':['constant','optimal','invscaling','adaptive']
        }
    name, mae = p(sgdg,sgd,'sgdreg')
    results['name'].append(name); results['mae'].append(mae)

    # Bayesian ARD regression.
    a=ARDRegression()
    gg={'alpha_1':[1e-3,1e-5,1e-7,1e-9],
    'alpha_2':[1e-3,1e-5,1e-7],
    'lambda_1':[1e-1,1e-3,1e-5,1e-7],
    'n_iter':[100,200,300],
    'lambda_2':[1e-3,1e-5,1e-7,1e-9],
    'tol':[1e-3,1e-5,1e-7,1e-9]}

    name, mae = p(gg,a,'ardreg')
    results['name'].append(name); results['mae'].append(mae)

    return results

In [15]:
# column transformation
xtrain,xtest,ytrain,ytest = split_data(merged_df_tr)
results_tr = check_models()

# column transformation + normalisation
xtrain,xtest,ytrain,ytest = split_data(merged_df_tr_minmax)
results_tr_minmax = check_models()

xtrain: (8133, 6) and xtest: (2712, 6)
ytrain: (8133,) and ytest: (2712,)
xtrain: (8133, 6) and xtest: (2712, 6)
ytrain: (8133,) and ytest: (2712,)


In [16]:
xtrain,xtest,ytrain,ytest = split_data(merged_df_tr_ohe)
results_tr_ohe = check_models()

xtrain: (8133, 28) and xtest: (2712, 28)
ytrain: (8133,) and ytest: (2712,)


In [50]:
# baseline
xtrain,xtest,ytrain,ytest = split_data(merged_df)
_,_,model = pt(RandomForestRegressor(),'randomforest')

xtrain: (8133, 6) and xtest: (2712, 6)
ytrain: (8133,) and ytest: (2712,)


In [66]:
# np.mean(results_tr['mae'])
# np.mean(results_tr_minmax['mae'])
((0.25 - 0.226)/0.25)*100

9.599999999999998

In [41]:
results_tr_ohe['name'] = ['model 1','model 2','model 3','model 4','model 5','model 6']
mae = results_tr_ohe.pop('mae')
results_tr_ohe['error'] = mae
results_tr_ohe

{'name': ['model 1', 'model 2', 'model 3', 'model 4', 'model 5', 'model 6'],
 'error': [0.22606579258931395,
  0.23613261062407698,
  0.2305432238117227,
  0.22592358967516915,
  0.2248809529681258,
  0.2252843366708819]}

In [74]:
features = [i.split("__")[0] for i in xtrain.columns]
importances = model.feature_importances_[:10]
indices = np.argsort(importances)[:10]
map_dict = dict(zip([i for i in range(len(features))],features))
feat_name = list(map(map_dict.get,indices))
ldf = pd.DataFrame({'name':feat_name,'fi':importances})

fig = px.pie(ldf.sort_values(by='fi'),names='name',values='fi',width=500,hole=.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

fig = px.area(results_tr_ohe,x='name',y='error',width=500,height=300,template='plotly_white',title='Model Accuracy')
fig.show()

# fig, ax = plt.subplots(figsize=(4,2))
# plt.title('Feature Importances')
# plt.barh(range(len(indices)), importances[indices], color='b', align='center')
# plt.yticks(range(len(indices)), [features[i] for i in indices])
# plt.xlabel('Relative Importance')
# plt.show()

In [44]:
px.bar(results_tr_ohe,x='name',y='error',width=500,height=300,template='plotly_white',title='relative accuracy')